In [1]:
import gurobipy as gp
#print(gp.gurobi.version())  # Print Gurobi version to verify installation
import pandas as pd

#comment out the code depending on the model 

import data_prep_Min
import modeling_Min
from modeling_Min import F, W, Y, I, m, df
#from modeling_Max import m, Y, W, Z


Set parameter Username
Set parameter LicenseID to value 2741742
Academic license - for non-commercial use only - expires 2026-11-19


In [2]:
m.update()
m.write("diversity_model.lp")

# Solve the model
m.optimize()

if m.status == gp.GRB.OPTIMAL:
    print(f"\nOptimal Solution Found - Total Penalty: {m.objVal:.1f}\n")
    
    # Print group assignments
    for f in F:
        print(f)
        if W[f].X > 0.5:  # If group f is used
            students_in_f = [i for i in I if Y[i,f].X > 0.5]
            students_in_f.sort()  # Sort student IDs
            print(f"Group {f}: {len(students_in_f)} students (IDs: {', '.join(map(str, students_in_f))})")
    
else:
    print(f"\nOptimization failed with status {m.status}")

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: Intel(R) Core(TM) 7 150U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 174 rows, 258 columns and 743 nonzeros (Min)
Model fingerprint: 0xb4fa8758
Model has 72 linear objective coefficients
Variable types: 0 continuous, 258 integer (222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 1710.0000000
Presolve removed 130 rows and 80 columns
Presolve time: 0.04s
Presolved: 44 rows, 178 columns, 536 nonzeros
Found heuristic solution: objective 1620.0000000
Variable types: 0 continuous, 178 integer (178 binary)

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 1620 171

In [5]:
def build_assignment_table():
    """
    Table: one row per student with their group and attributes.
    """
    rows = []

    def clean_col_name(col):
        # Flatten multi-index column names if necessary
        if isinstance(col, tuple):
            # Remove unnamed parts
            parts = [p for p in col if p is not None and not str(p).startswith("Unnamed")]
            name = parts[-1] if parts else col[0]
        else:
            name = col

        name = str(name).strip()
        lname = name.lower()

        # Columns Names
        if lname == "Person ID":
            return "Person_ID"
        if lname == "lived experience?":
            return "Lived_Experience?"
        if lname == "Minnesota?":
            return "Minnesota_Resident?"
        if "can't be assigned with" in lname:
            return "Can't be assigned with"
        
        return name
          
    for i in I:
        # Find assigned group
        assigned_group = None
        for f in F:
            if Y[i, f].X > 0.5:
                assigned_group = f
                break

        row = {
            "Person_ID": i,
            "Group": assigned_group
        }

        # Attach original data columns that are helpful
        # Add anything else you want here
        for col in df.columns:
            new_name = clean_col_name(col)
            
            if new_name == "Person_ID":
                continue  # Already have this

            row[new_name] = df.loc[i, col]

        rows.append(row)

    assign_df = pd.DataFrame(rows)

    # Remove person ID column if not needed
    assign_df = assign_df.drop(columns=["Person_ID"], errors='ignore')
   
    # Make group numbers start at 1 instead of 0
    assign_df["Group"] = assign_df["Group"].astype("Int64") + 1

    # Replace NaN with blank for can't be assigned with column
    assign_df = assign_df.fillna("")

    # Sort by group then student
    assign_df = assign_df.sort_values(["Group", "Person ID"]).reset_index(drop=True)
    return assign_df

build_assignment_table()

,Group,Person ID,Social Science,Computational/Math,Real World,Lived_Experience?,Minnesota?,Can't be assgned with
0,1,1,0,1,0,0,0,
1,1,14,1,0,0,0,0,
2,1,25,0,0,1,1,1,
3,1,26,0,0,1,1,1,
4,1,29,0,0,1,0,1,
5,2,4,1,0,0,0,0,9.0
6,2,6,0,1,0,0,0,
7,2,15,0,0,1,1,1,
8,2,17,0,0,1,1,1,
9,2,28,1,0,0,0,1,


In [6]:
def build_group_summary_table():
    """
    Table: one row per group with size and lists Person_IDs.
    """
    rows = []
    for f in F:
        members = [i for i in I if Y[i, f].X > 0.5]
        size_f = len(members)

        if size_f == 0:
            # skip completely unused groups to keep table clean
            continue

        rows.append({
            "Group": f + 1,        # 1-based
            "Size": size_f,
            "Person_IDs": members
        })

    group_df = pd.DataFrame(rows).sort_values("Group").reset_index(drop=True)
    return group_df

build_group_summary_table()

,Group,Size,Person_IDs
0,1,5,"[0, 13, 24, 25, 28]"
1,2,6,"[3, 5, 14, 16, 27, 29]"
2,3,6,"[2, 4, 6, 9, 23, 26]"
3,4,4,"[11, 12, 20, 21]"
4,5,4,"[7, 8, 18, 22]"
5,6,5,"[1, 10, 15, 17, 19]"
